In [47]:
import numpy as np


class KMeans():

    '''
        Class KMeans:
        Attr:
            n_cluster - Number of cluster for kmeans clustering (Int)
            max_iter - maximum updates for kmeans clustering (Int) 
            e - error tolerance (Float)
    '''

    def __init__(self, n_cluster, max_iter=100, e=0.0001):
        self.n_cluster = n_cluster
        self.max_iter = max_iter
        self.e = e

    def fit(self, x):
        '''
            Finds n_cluster in the data x
            params:
                x - N X D numpy array
            returns:
                A tuple
                (centroids a n_cluster X D numpy array,
                y a size (N,) numpy array where cell i is the ith sample's assigned cluster,
                number_of_updates an Int)
            Note: Number of iterations is the number of time you update the assignment
        ''' 
        assert len(x.shape) == 2, "fit function takes 2-D numpy arrays as input"
        np.random.seed(42)
        N, D = x.shape

        # TODO:
        # - comment/remove the exception.
        # - Initialize means by picking self.n_cluster from N data points
        # - Update means and membership until convergence or until you have made self.max_iter updates.
        # - return (means, membership, number_of_updates)

        # DONOT CHANGE CODE ABOVE THIS LINE
        #raise Exception(
        #    'Implement fit function in KMeans class (filename: kmeans.py)')
        # initialize mu and J
        mu0_idx = np.random.choice(range(N), self.n_cluster)
        mu = x[mu0_idx,:]
        j = 1e10

        def compute_ridx(x, mu):
            A = np.sum(x**2,axis=1).reshape(x.shape[0],1)
            B = np.sum(mu**2,axis=1).reshape(mu.shape[0],1)
            AB = np.dot(x, np.transpose(mu))
            dists = np.sqrt(-2*AB + A + np.transpose(B))
    
            # because r is {0,1}, just return index 
            r = np.argmin(np.transpose(dists), axis=1)
            return np.ravel(r)



        for itr in range(self.max_iter):    
    
            r_idx = compute_ridx(mu, x)
    
            # r_ik = 0 if xi not in k
            # mu[r_idx,:] means mu_k when r_ik = 1, size N X D
            #j1 = sum(np.sum(np.multiply((x - mu[r_idx,:]), (x - mu[r_idx,:])), axis=1))
            j2 = np.sum(np.multiply((mu[r_idx,:] - x ), (mu[r_idx,:] - x)))
            #print(j-j2)
            
            # stop condition
            if abs(j-j2) < self.e:
                break
            
            j = j2
            # r_idx is asigned index for each class
            mu = np.array(
                [np.average(x[r_idx==i,:], axis=0) for i in range(self.n_cluster)])
            
        y = compute_ridx(mu, x)
        return (mu, y, itr)
        
        # DONOT CHANGE CODE BELOW THIS LINE

class KMeansClassifier():

    '''
        Class KMeansClassifier:
        Attr:
            n_cluster - Number of cluster for kmeans clustering (Int)
            max_iter - maximum updates for kmeans clustering (Int) 
            e - error tolerance (Float) 
    '''

    def __init__(self, n_cluster, max_iter=100, e=1e-6):
        self.n_cluster = n_cluster
        self.max_iter = max_iter
        self.e = e

    def fit(self, x, y):
        '''
            Train the classifier
            params:
                x - N X D size  numpy array
                y - (N,) size numpy array of labels
            returns:
                None
            Stores following attributes:
                self.centroids : centroids obtained by kmeans clustering (n_cluster X D numpy array)
                self.centroid_labels : labels of each centroid obtained by 
                    majority voting ((N,) numpy array) 
        '''

        assert len(x.shape) == 2, "x should be a 2-D numpy array"
        assert len(y.shape) == 1, "y should be a 1-D numpy array"
        assert y.shape[0] == x.shape[0], "y and x should have same rows"

        np.random.seed(42)
        N, D = x.shape
        # TODO:
        # - comment/remove the exception.
        # - Implement the classifier
        # - assign means to centroids
        # - assign labels to centroid_labels

        # DONOT CHANGE CODE ABOVE THIS LINE
        #raise Exception(
        #    'Implement fit function in KMeansClassifier class (filename: kmeans.py)')
        
        kmeans = KMeans(self.n_cluster, self.max_iter, self.e) ## self.n ....
        centroids, membership, i = kmeans.fit(x)
        
        centroid_labels = np.zeros((n_cluster,)) # self.n_cluster
        for k in range(n_cluster): # self.n_cluster
            centroid_labels[k] = np.argmax(np.bincount(y[membership==k]))        
        
        self.centroids = centroids
        self.centroid_labels = centroid_labels

        # DONOT CHANGE CODE BELOW THIS LINE

        assert self.centroid_labels.shape == (self.n_cluster,), 'centroid_labels should be a numpy array of shape ({},)'.format(
            self.n_cluster)

        assert self.centroids.shape == (self.n_cluster, D), 'centroid should be a numpy array of shape {} X {}'.format(
            self.n_cluster, D)

    def predict(self, x):
        '''
            Predict function

            params:
                x - N X D size  numpy array
            returns:
                predicted labels - numpy array of size (N,)
        '''

        assert len(x.shape) == 2, "x should be a 2-D numpy array"

        np.random.seed(42)
        N, D = x.shape
        # TODO:
        # - comment/remove the exception.
        # - Implement the prediction algorithm
        # - return labels

        # DONOT CHANGE CODE ABOVE THIS LINE
        #raise Exception(
        #    'Implement predict function in KMeansClassifier class (filename: kmeans.py)')
        
        def compute_ridx(x, mu):
            # input (N X D) ,(C X D)
            # output (C X 1)
            A = np.sum(x**2,axis=1).reshape(x.shape[0],1)
            B = np.sum(mu**2,axis=1).reshape(mu.shape[0],1)
            AB = np.dot(x, np.transpose(mu))
            dists = np.sqrt(-2*AB + A + np.transpose(B))
    
            # because r is {0,1}, just return index 
            r = np.argmin(np.transpose(dists), axis=1)
            return np.ravel(r)
        
        mincent = compute_ridx(self.centroids, x) # self.centroids
        labels = self.centroid_labels[list(mincent)] # self.centroids_labels
        
        # DONOT CHANGE CODE BELOW THIS LINE
        return labels



In [2]:
import numpy as np
from data_loader import toy_dataset, load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from utils import Figure

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
x, y = toy_dataset(4)


### 1.1

In [3]:
#type(x)
N, D = x.shape
print(N,D)

200 2


In [4]:
np.random.seed(42)

n_cluster = 5
max_iter = 100
e = 0.0001

In [5]:
# initialize mu and J
mu0_idx = np.random.choice(range(N), n_cluster)
mu = x[mu0_idx,:]
j = 1e15

def compute_ridx(x, mu):
    A = np.sum(x**2,axis=1).reshape(x.shape[0],1)
    B = np.sum(mu**2,axis=1).reshape(mu.shape[0],1)
    AB = np.dot(x, np.transpose(mu))
    dists = np.sqrt(-2*AB + A + np.transpose(B))
    
    # because r is {0,1}, just return index 
    r = np.argmin(np.transpose(dists), axis=1)
    return np.ravel(r)


for itr in range(100):
    print(mu[0,:])
    
    
    r_idx = compute_ridx(mu, x)
    
    # r_ik = 0 if xi not in k
    # mu[r_idx,:] means mu_k when r_ik = 1, size N X D
    #j1 = sum(np.sum(np.multiply((x - mu[r_idx,:]), (x - mu[r_idx,:])), axis=1))
    j2 = np.sum(np.multiply((mu[r_idx,:] - x ), (mu[r_idx,:] - x)))
    if abs(j-j2) < e:
        break
    j = j2
    # r_idx
    mu = np.array([np.average(x[r_idx==i,:], axis=0) for i in range(n_cluster)])
print(mu[0,:])
y = compute_ridx(mu, x)
print(y[10:11])
print(y.shape)
print(itr)
retup = (mu, y, itr)
print(type(retup))

[-5.37766937 -0.93782504]
[-4.83651121 -0.36487946]
[-4.88316325 -0.13992194]
[-4.89069392  0.0557828 ]
[-4.84703256  0.08214575]
[-4.84703256  0.08214575]
[-4.84703256  0.08214575]
[3]
(200,)
5
<class 'tuple'>


C:\Users\Takato\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in sqrt
  # Remove the CWD from sys.path while we load stuff.


In [6]:
ma = np.array([[1,1],[2,2]])
xb = np.array([[4,5],[2,2],[0,2],[0,0],[3,3]])
A = np.sum(ma**2,axis=1).reshape(2,1)
B = np.sum(xb**2,axis=1).reshape(5,1)
AB = np.dot(ma,np.transpose(xb))
dists = np.sqrt(-2*AB + A + np.transpose(B))
dists

array([[5.        , 1.41421356, 1.41421356, 1.41421356, 2.82842712],
       [3.60555128, 0.        , 2.        , 2.82842712, 1.41421356]])

In [7]:
r = compute_ridx(ma, xb)
r

array([1, 1, 0, 0, 1], dtype=int64)

In [8]:
for i in list(r):
    print(ma[i,:])

[2 2]
[2 2]
[1 1]
[1 1]
[2 2]


In [9]:
ma[r,:]

array([[2, 2],
       [2, 2],
       [1, 1],
       [1, 1],
       [2, 2]])

In [10]:
print(ma[r,:])
print(xb - ma[r,:])
np.sum(np.multiply((xb - ma[r,:]), (xb - ma[r,:])), axis=1)

[[2 2]
 [2 2]
 [1 1]
 [1 1]
 [2 2]]
[[ 2  3]
 [ 0  0]
 [-1  1]
 [-1 -1]
 [ 1  1]]


array([13,  0,  2,  2,  2])

In [11]:
np.sum(np.multiply(xb, xb), axis=1)

array([41,  8,  4,  0, 18])

In [12]:
[i for i in set(list(r))]

[0, 1]

In [13]:
xb[r==1,:]

array([[4, 5],
       [2, 2],
       [3, 3]])

In [14]:
np.average(xb[r==1,:], axis=0)

array([3.        , 3.33333333])

In [15]:
[np.average(xb[r==i,:], axis=0) for i in set(list(r))]

[array([0., 1.]), array([3.        , 3.33333333])]

In [16]:
np.array([np.average(xb[r==i,:], axis=0) for i in set(list(r))])

array([[0.        , 1.        ],
       [3.        , 3.33333333]])

### 1.2

In [13]:
imx = np.array([[1,10,1],[40,20,90],[80,45,15],[80,45,15]])
immu = np.array([13,44,77])

ax0, ax1 = imx.shape
print(ax0, ax1)
imx0 = np.ravel(imx)
imxre = imx0.reshape((ax0, ax1))
print(imxre)

4 3
[[ 1 10  1]
 [40 20 90]
 [80 45 15]
 [80 45 15]]


In [29]:
def nearest_value(mu, targetn):
    # mu: K X 1 numpy array
    # targetn: scalar
    # reterun nearest value
    idx = np.abs(mu - targetn).argmin()
    return mu[idx]

print(nearest_value(immu, 30))

44


In [30]:
np.array([nearest_value(immu, i) for i in list(imx0)]).reshape((ax0, ax1))

array([[13, 13, 13],
       [44, 13, 77],
       [77, 44, 13],
       [77, 44, 13]])

In [24]:
z2 = np.zeros((2,2,2))

z2a , z2b = z2.shape[0:2]
z2a

2

### 1.3

In [5]:
    x_train, x_test, y_train, y_test = load_digits()

    # plot some train data
    N = 25
    l = int(np.ceil(np.sqrt(N)))

    im = np.zeros((10 * l, 10 * l))
    for m in range(l):
        for n in range(l):
            if (m * l + n < N):
                im[10 * m:10 * m + 8, 10 * n:10 * n +
                    8] = x_train[m * l + n].reshape([8, 8])
    plt.imsave('plots/digits.png', im, cmap='Greys')

    n_cluster = 30
    max_iter=100
    e=1e-6
    #classifier = KMeansClassifier(n_cluster=n_cluster, max_iter=100, e=1e-6)

    #classifier.fit(x_train, y_train)
    #y_hat_test = classifier.predict(x_test)


In [30]:
        #def fit(self, x, y):
        '''
            Train the classifier
            params:
                x - N X D size  numpy array
                y - (N,) size numpy array of labels
            returns:
                None
            Stores following attributes:
                self.centroids : centroids obtained by kmeans clustering (n_cluster X D numpy array)
                self.centroid_labels : labels of each centroid obtained by 
                    majority voting ((N,) numpy array) 
        '''
        x = x_train
        y = y_train

        np.random.seed(42)
        N, D = x.shape
        
        # TODO:
        # - comment/remove the exception.
        # - Implement the classifier
        # - assign means to centroids
        # - assign labels to centroid_labels

        # DONOT CHANGE CODE ABOVE THIS LINE
        #raise Exception(
        #    'Implement fit function in KMeansClassifier class (filename: kmeans.py)')
        kmeans = KMeans(n_cluster, max_iter, e) ## self.n ....
        centroids, membership, i = kmeans.fit(x)
        
        centroid_labels = np.zeros((n_cluster,)) # self.n_cluster
        for k in range(n_cluster): # self.n_cluster
            centroid_labels[k] = np.argmax(np.bincount(y[membership==k]))
            
        
        
        

        # DONOT CHANGE CODE BELOW THIS LINE

In [15]:
x.shape

(1347, 64)

In [10]:
centroids.shape

(30, 64)

In [12]:
membership.shape

(1347,)

In [14]:
set(membership)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29}

In [13]:
set(y)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [28]:
np.zeros((3,))

array([0., 0., 0.])

In [29]:
np.zeros((3,)).shape

(3,)

In [24]:
y[membership==0]

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4])

In [25]:
np.argmax(y[membership==0])

0

In [26]:
 np.bincount(y[membership==0])

array([ 0,  0,  0,  0, 30])

In [27]:
np.argmax(np.bincount(y[membership==0]))

4

In [31]:
centroid_labels

array([4., 0., 4., 2., 2., 8., 8., 7., 1., 3., 0., 5., 8., 1., 5., 8., 5.,
       1., 6., 3., 5., 2., 6., 8., 4., 6., 7., 1., 1., 9.])

In [41]:
        x = x_test
        #def predict(self, x):
        '''
            Predict function

            params:
                x - N X D size  numpy array
            returns:
                predicted labels - numpy array of size (N,)
        '''

        assert len(x.shape) == 2, "x should be a 2-D numpy array"

        np.random.seed(42)
        N, D = x.shape
        # TODO:
        # - comment/remove the exception.
        # - Implement the prediction algorithm
        # - return labels
        
        def compute_ridx(x, mu):
            # input (N X D) ,(C X D)
            # output (C X 1)
            A = np.sum(x**2,axis=1).reshape(x.shape[0],1)
            B = np.sum(mu**2,axis=1).reshape(mu.shape[0],1)
            AB = np.dot(x, np.transpose(mu))
            dists = np.sqrt(-2*AB + A + np.transpose(B))
    
            # because r is {0,1}, just return index 
            r = np.argmin(np.transpose(dists), axis=1)
            return np.ravel(r)
        mincent = compute_ridx(centroids, x) # self.centroids
        centroid_labels[list(mincent)] # self.centroids_labels
        
        

        # DONOT CHANGE CODE ABOVE THIS LINE
        #raise Exception(
        #    'Implement predict function in KMeansClassifier class (filename: kmeans.py)')
        # DONOT CHANGE CODE BELOW THIS LINE
        #return labels


In [42]:
mincent.shape

(450,)

In [43]:
set(mincent)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29}

In [37]:
x.shape

(450, 64)

In [44]:
centroid_labels[[1,2,5]]

array([0., 4., 8.])

In [45]:
centroid_labels[list(mincent)]

array([6., 9., 3., 7., 2., 1., 5., 2., 5., 2., 1., 1., 4., 0., 4., 2., 3.,
       7., 8., 8., 4., 3., 9., 7., 5., 6., 3., 5., 6., 3., 4., 9., 1., 4.,
       4., 6., 9., 4., 7., 6., 6., 9., 1., 3., 6., 1., 3., 0., 6., 5., 5.,
       1., 9., 5., 6., 0., 3., 0., 0., 1., 0., 4., 5., 2., 4., 5., 7., 0.,
       7., 5., 9., 9., 5., 4., 7., 0., 4., 5., 5., 9., 9., 0., 2., 3., 8.,
       0., 6., 4., 4., 9., 1., 2., 8., 3., 5., 2., 9., 4., 4., 4., 4., 3.,
       5., 3., 1., 8., 5., 1., 4., 2., 7., 7., 4., 4., 4., 9., 8., 7., 8.,
       7., 2., 6., 9., 4., 0., 7., 2., 7., 5., 8., 7., 5., 7., 7., 0., 6.,
       6., 4., 2., 8., 0., 9., 4., 6., 8., 1., 6., 9., 0., 3., 5., 6., 6.,
       0., 6., 4., 2., 9., 3., 8., 7., 2., 9., 0., 4., 5., 8., 6., 5., 7.,
       8., 1., 4., 2., 1., 3., 7., 7., 2., 2., 3., 9., 8., 0., 3., 2., 2.,
       5., 6., 9., 9., 4., 1., 5., 4., 2., 9., 6., 4., 1., 5., 9., 5., 7.,
       1., 9., 4., 8., 1., 5., 4., 4., 9., 6., 1., 8., 6., 0., 4., 5., 2.,
       7., 4., 6., 4., 5.

In [46]:
centroid_labels[list(mincent)].shape

(450,)

## ~ Test ~

In [33]:
import numpy as np
from data_loader import toy_dataset, load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from utils import Figure

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt


def transform_image(image, code_vectors):
    '''
        Quantize image using the code_vectors

        Return new image from the image by replacing each RGB value in image with nearest code vectors (nearest in euclidean distance sense)

        returns:
            numpy array of shape image.shape
    '''

    assert image.shape[2] == 3 and len(image.shape) == 3, \
        'Image should be a 3-D array with size (?,?,3)'

    assert code_vectors.shape[1] == 3 and len(code_vectors.shape) == 2, \
        'code_vectors should be a 2-D array with size (?,3)'

    # TODO
    # - comment/remove the exception
    # - implement the function

    # DONOT CHANGE CODE ABOVE THIS LINE
    #raise Exception(
    #    'Implement transform_image function (filename:kmeansTest.py)')
    
    def nearest_value(mu, targetn):
        # mu: K X 1 numpy array
        # targetn: scalar
        # reterun nearest value
        idx = np.abs(mu - targetn).argmin()
        return mu[idx]
    
    ax0, ax1, ax2 = image.shape
    new_im = np.zeros((ax0, ax1, ax2))
    
    # RGB 3 dimension for loop
    for k in range(3):
        #print(k)
        # reshape 1D
        imx0 = np.ravel(image[:,:,k])
        
        new_im[:,:,k] = np.array([nearest_value(code_vectors[:,k], i) for i in list(imx0)]).reshape((ax0, ax1)) 
    
    
    # DONOT CHANGE CODE BELOW THIS LINE
    return new_im

################################################################################
# KMeans on 2D toy dataset
# The dataset is generated from N gaussian distributions equally spaced on N radius circle.
# Here, N=4
# KMeans on this dataset should be able to identify the 4 clusters almost clearly.
################################################################################


def kmeans_toy():
    x, y = toy_dataset(4)
    fig = Figure()
    fig.ax.scatter(x[:, 0], x[:, 1], c=y)
    fig.savefig('plots/toy_dataset_real_labels.png')

    fig.ax.scatter(x[:, 0], x[:, 1])
    fig.savefig('plots/toy_dataset.png')
    n_cluster = 4
    k_means = KMeans(n_cluster=n_cluster, max_iter=100, e=1e-8)
    centroids, membership, i = k_means.fit(x)

    assert centroids.shape == (n_cluster, 2), \
        ('centroids for toy dataset should be numpy array of size {} X 2'
            .format(n_cluster))

    assert membership.shape == (50 * n_cluster,), \
        'membership for toy dataset should be a vector of size 200'

    assert type(i) == int and i > 0,  \
        'Number of updates for toy datasets should be integer and positive'

    print('[success] : kmeans clustering done on toy dataset')
    print('Toy dataset K means clustering converged in {} steps'.format(i))

    fig = Figure()
    fig.ax.scatter(x[:, 0], x[:, 1], c=membership)
    fig.ax.scatter(centroids[:, 0], centroids[:, 1], c='red')
    fig.savefig('plots/toy_dataset_predicted_labels.png')

    np.savez('results/k_means_toy.npz',
             centroids=centroids, step=i, membership=membership, y=y)

################################################################################
# KMeans for image compression
# Here we use k-means for compressing an image
# We load an image 'baboon.tiff',  scale it to [0,1] and compress it.
# The problem can be rephrased as --- "each pixel is a 3-D data point (RGB) and we want to map each point to N points or N clusters.
################################################################################


def kmeans_image_compression():
    im = plt.imread('baboon.tiff')
    N, M = im.shape[:2]
    im = im / 255

    # convert to RGB array
    data = im.reshape(N * M, 3)

    k_means = KMeans(n_cluster=16, max_iter=100, e=1e-6)
    centroids, _, i = k_means.fit(data)

    print('RGB centroids computed in {} iteration'.format(i))
    new_im = transform_image(im, centroids)

    assert new_im.shape == im.shape, \
        'Shape of transformed image should be same as image'

    mse = np.sum((im - new_im)**2) / (N * M)
    print('Mean square error per pixel is {}'.format(mse))
    plt.imsave('plots/compressed_baboon.png', new_im)

    np.savez('results/k_means_compression.npz', im=im, centroids=centroids,
             step=i, new_image=new_im, pixel_error=mse)


################################################################################
# Kmeans for classification
# Here we use k-means for classifying digits
# We find N clusters in the data and label each cluster with the maximal class that belongs to that cluster.
# Test samples are labeled based on which cluster they belong to
################################################################################

def kmeans_classification():
    x_train, x_test, y_train, y_test = load_digits()

    # plot some train data
    N = 25
    l = int(np.ceil(np.sqrt(N)))

    im = np.zeros((10 * l, 10 * l))
    for m in range(l):
        for n in range(l):
            if (m * l + n < N):
                im[10 * m:10 * m + 8, 10 * n:10 * n +
                    8] = x_train[m * l + n].reshape([8, 8])
    plt.imsave('plots/digits.png', im, cmap='Greys')

    n_cluster = 30
    classifier = KMeansClassifier(n_cluster=n_cluster, max_iter=100, e=1e-6)

    classifier.fit(x_train, y_train)
    y_hat_test = classifier.predict(x_test)

    assert y_hat_test.shape == y_test.shape, \
        'y_hat_test and y_test should have same shape'

    print('Prediction accuracy of K-means classifier with {} cluster is {}'.
          format(n_cluster, np.mean(y_hat_test == y_test)))

    linear_classifier = LogisticRegression()
    linear_classifier.fit(x_train, y_train)
    y_hat_test = linear_classifier.predict(x_test)
    print('Accuracy of logistic regression classifier is {}'
          .format(np.mean(y_hat_test == y_test)))

    KNNClassifier = KNeighborsClassifier()
    KNNClassifier.fit(x_train, y_train)
    y_hat_test = KNNClassifier.predict(x_test)
    print('Accuracy of Nearest Neighbour classifier is {}'
          .format(np.mean(y_hat_test == y_test)))

    np.savez('results/k_means_classification.npz',
             y_hat_test=y_hat_test, y_test=y_test, centroids=classifier.centroids, centroid_labels=classifier.centroid_labels)


if __name__ == '__main__':
#    kmeans_toy()
    kmeans_image_compression()
#    kmeans_classification()


RGB centroids computed in 99 iteration
Mean square error per pixel is 0.0023375327639532186
